In [1]:
import pandas as pd
import geopandas as gpd
from shapely import Point, LineString, MultiLineString
from geopy.geocoders import Nominatim
import requests
import networkx as nx
from shapely.wkt import loads
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import to_hex
import dash_bootstrap_components as dbc
from dash_extensions.enrich import DashProxy, MultiplexerTransform
import dash_leaflet as dl
from dash import html, dcc
import plotly.express as px
from dash.dependencies import Input, Output, State
import webbrowser
import random
import pickle
import osmnx as ox
import warnings
warnings.filterwarnings("ignore")

In [2]:
PRIVATE_JET_OCUP = 50 #Estimated based on 
PRIVATE_JET_EF = 3.044726409584053 # 4.9 kg per mile / 1.60934 km/mile(https://flybitlux.com/what-is-the-carbon-footprint-of-a-private-jet/#:~:text=A%20typical%20private%20jet%20emits,grams%20per%20passenger%20per%20kilometer.)

In [3]:
emission_comparisson = [
    (1, ' times the CO2 an average European emits in a month'), 
    (1, ' flights from London to New York'), 
    (1961, ' vegetarian meals'), 
    (138, ' meat-based meals'),
    (1 ,' years of trash produced by a household in Canada'), 
    (4, ' months of energy for heating a home in Canada'),
    (72, ' trains from Amsterdam to Paris')
]

In [4]:
def geolocate(address1=None, address2=None):
    geolocator = Nominatim(user_agent="xxx")
    if address2 and address1:
        address = f'{address1}, {address2}'
    elif address1:
        address = address1
    elif address2:
        address = address2
    else:
        print('No address provided')
        return None
    try:
        address = geolocator.geocode(address, timeout=10)
        if address:
            coords = [address.latitude,address.longitude]
            return coords
        else:
            try:
                address = geolocator.geocode(address1, timeout=10)
                if address:
                    coords = [address.latitude,address.longitude]
                    return coords
                else:
                    print('error:',address)
                    return []
            except:
                print('error:',address)
                return []
    except:
        print('error:',address)
        return []
    
def get_capital(country):
    base_url = "https://restcountries.com/v3.1/name/"

    try:
        response = requests.get(f"{base_url}{country}")
        response.raise_for_status()  # Raise an HTTPError for bad responses

        data = response.json()
        capital = data[0]['capital'][0]

        return capital
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return country
    
def find_path(G, source, target, method='min_connections'):
    if method == 'min_connections':
        try:
            return nx.shortest_path(G, source, target)
        except:
            return []
#             print('error:',source,target)
    elif method == 'min_length':
        try:
            return nx.dijkstra_path(G, source, target, weight='Distance')
        except:
            return []
#             print('error:',source,target)
    else:
        print('Invalid Method')
        
        
def shortest_path_edges(path):

    edges = []
    try:
        for node in range(len(path)-1):
            O = path[node]
            D = path[node+1]
            edges.append([O,D])
    except:
        edges.append([])
    return edges

def wkt_linestring_to_list_of_lists(wkt_linestring):
    linestring = loads(wkt_linestring)
    coordinates = list(linestring.coords)
    return coordinates

def normalize_column(column, ascending=True):
    norm =  (column - column.min()) / (column.max() - column.min())
    if ascending:
        return norm
    else:
        return 1-norm
    
def estimate_ls(country_col,country_val,df,y,X,return_r2=False):
    country_df = df[df[country_col]==country_val]
    model = LinearRegression() 
    model.fit(country_df[X].values,country_df[y].values)
    if not return_r2:
        return model
    else:
        y_pred = model.predict(country_df[X].values)
        r2 = r2_score(country_df[y].values, y_pred)
        return r2

def extract_colors(palette_name, n_colors, return_rgb=True):
    try:
        # Get the colormap
        cmap = plt.get_cmap(palette_name)
        
        # Generate n equally spaced values between 0 and 1
        values = np.linspace(0, 1, n_colors)
        
        # Get the colors from the colormap at the specified values
        rgb_colors = cmap(values)
        if return_rgb:
            return rgb_colors
        else:
            # Convert RGB to hex
            hex_colors = [to_hex(color) for color in rgb_colors]
            return hex_colors
        
    except ValueError as e:
        print(f"Error: {e}")
        return None
    
def apply_prioritization(df):
    #Multicriteria Prioritization

    df['CO2EmPC/Year_Norm'] = df.groupby('COP')['CO2EmPC/Year'].transform(normalize_column)
    df['CO2Em/Year_Norm'] = df.groupby('COP')['CO2Em/Year'].transform(normalize_column)
    df['Assistance_Norm'] = df.groupby('COP')['Assistance'].transform(normalize_column)
    df['Population_Norm'] = df.groupby('COP')['Population'].transform(normalize_column)
    df['Distance_Norm'] = df.groupby('COP')['Distance'].transform(normalize_column)
    df['FlightCO2Em_Norm'] = df.groupby('COP')['FlightCO2Em'].transform(normalize_column)
    
    df['Coef'] = (
        (CO2EMPC_COEF  * df['CO2EmPC/Year_Norm']) + \
        (CO2EM_COEF    * df['CO2Em/Year_Norm'])   + \
        (POP_COEF      * df['Population_Norm'])   + \
        (ASSIST_COEF   * df['Assistance_Norm'])   + \
        (DIST_COEF     * df['Distance_Norm'])     + \
        (FLICO2EM_COEF * df['FlightCO2Em_Norm'])
    )

    df['Coef'] = df.apply(lambda row: row['Coef'] if row['GuestCountry']!=row['HostCountry'] else 0,axis=1)
    df['Coef'] = df.groupby('COP')['Coef'].transform(normalize_column)

    df['Hierarchy'] = df.groupby('COP')['Coef'].rank(ascending=False).astype(int)
    return(df)

def extract_linestrings(geometry):
    if isinstance(geometry, MultiLineString):
        return list(geometry.geoms)
    else:
        return [geometry]

def estimate_cop(COP, year, hostCountry, hostCity, historic_df, assistance_factor=1, emissions_factor=1, shortest_path_method='min_length', local=False):
    hostCity = get_capital(hostCountry)
    host_df = historic_df[historic_df['GuestCountry']==hostCountry][
        ['GuestCountry','GuestContinent','GuestAirportID','GuestAirportNam','GuestAirportCity','GuestIATA','GuestICAO']
    ].drop_duplicates()

    host_df = host_df.rename(
        columns={col:col.replace('Guest','Host') for col in host_df.columns}
    ).T

    host_df = host_df[host_df.columns[0]]

#     guests_df = historic_df[
#         ['GuestCountry','GuestContinent','GuestAirportID','GuestAirportNam','GuestAirportCity','GuestIATA','GuestICAO']
#     ].drop_duplicates()

    #maintain population and assistance from COP28
    guests_df = historic_df[
        ['GuestCountry','GuestContinent','GuestAirportID','GuestAirportNam','GuestAirportCity','GuestIATA','GuestICAO']
    ].drop_duplicates()

    for key,value in host_df.items():
        guests_df[key] = value

    guests_df['COP'] = COP
    guests_df['Year'] = year

    guests_df['shortest_path'] = guests_df.apply(
        lambda row: find_path(G, row['GuestAirportID'],row['HostAirportID'], method=shortest_path_method), axis=1
    )

    guests_df = guests_df[guests_df.shortest_path.apply(lambda x: len(x))!= 0]

    guests_df['shortest_path_edges'] = guests_df['shortest_path'].apply(
        shortest_path_edges
    ).apply(lambda x: [[x.index(c),c] for c in x])

    df_flights = guests_df.explode('shortest_path_edges')

    df_flights['path_index'] = df_flights['shortest_path_edges'].apply(lambda x: x[0] if type(x)==list else np.nan)
    df_flights['SAirportID'] = df_flights['shortest_path_edges'].apply(lambda x: x[1][0] if type(x)==list else np.nan)
    df_flights['DAirportID'] = df_flights['shortest_path_edges'].apply(lambda x: x[1][1] if type(x)==list else np.nan)

    df_flights = gpd.GeoDataFrame(df_flights.merge(routes_grouped[['SAirportID', 'DAirportID', 'Distance', 'geometry']]), crs=4326)

    guests_df = guests_df.merge(
        df_flights.groupby(['COP','GuestCountry']).agg({'Distance':'sum','geometry':list}).reset_index(),
        left_on=['COP','GuestCountry'], right_on=['COP','GuestCountry'],how='left'
    )

    guests_df['geometry'] = guests_df['geometry'].apply(
        lambda x: MultiLineString(x) if type(x) == list else np.nan
    )
    
    guests_df = gpd.GeoDataFrame(guests_df,crs=4326)

    guests_df = guests_df[guests_df['GuestCountry']!='European Union']
    
    guests_df['Distance'] = guests_df.apply(lambda row: row['Distance'] if row['GuestCountry']!=row['HostCountry'] else 0,axis=1).fillna(0)

    #Predictions   
    guests_df = guests_df.merge(countries)
    guests_df['Population'] = guests_df.apply(
        lambda row: row['PopulationModel'].predict(np.array(row['Year']).reshape(-1, 1))[0],
        axis=1
    ).fillna(0)
    
    guests_df['CO2Em/Year'] = guests_df.apply(
        lambda row: row['EmissionsModel'].predict(np.array(row['Year']).reshape(-1, 1))[0],
        axis=1
    ).fillna(0)
    
    guests_df['CO2EmPC/Year'] = (guests_df['CO2Em/Year']/guests_df['Population']).fillna(0)
    
    guests_df['Assistance'] = guests_df.apply(
        lambda row: max(
            row['AssistanceModel'].predict(
                np.array(row[['Year','CO2Em/Year','Distance']]).reshape(1,-1)
            )[0],
            0
        ),
        axis=1
    )

    guests_df['Flights'] = guests_df['Assistance'] / PRIVATE_JET_OCUP
    guests_df['FlightCO2Em'] = (guests_df['Flights'] * (guests_df['Distance']/1000) * PRIVATE_JET_EF)/1000
    guests_df['HostCity'] = hostCity
    
    
#     #Multicriteria Prioritization

#     df = apply_prioritization(guests_df)
    
    return guests_df

def create_line_chart(df, x, y,text=None):

    # Create the figure with Plotly Express
    fig=px.line(
        df, 
        x=x, 
        y=y, 
        labels={'value': 'Flight Emissions due to COP (ton CO2)', 'variable': 'Scenario'},
        color_discrete_map={'Historic':cop_colors['COP27'],'Projected': cop_colors['COP25']},
        markers=True,
        text=text
    )

    # Update the layout of the figure only
    fig.update_layout(
        legend=dict(orientation="h", y=1.1, x=0.5, xanchor="center"),  # Set orientation to horizontal and adjust position
        margin=dict(l=10, r=10, t=30, b=10),  # Adjust margin/padding
        font=dict(family="Montserrat", color='white',size=8),  # Adjust font
        paper_bgcolor='#212222',
        
    )
    
    if text is not None:
        fig.update_traces(textposition='top center', textfont=dict(size=10,color=colorscale[0]))
    
    return fig

def imagine_local_cop(location, known_location=None):
    try:
        osmG = ox.graph_from_place(location, simplify=True, retain_all=False, network_type="drive")
    except:
        osmG = ox.graph_from_point(geolocate(location),dist=6000,simplify=True, retain_all=False, network_type="drive")

    #Hotels
    tags = {"tourism": "hotel"}
    try:
        hotels = ox.features_from_place(location, tags, which_result=None, buffer_dist=None).reset_index()
    except:
        hotels = ox.features_from_point(geolocate(location),tags,6000)
    hotels['geometry'] = hotels['geometry'].apply(lambda geom: geom if geom.type=='Point' else geom.centroid)
    hotels['nearest_node'] = ox.nearest_nodes(osmG,hotels.geometry.x,hotels.geometry.y)
    hotels['Type'] = 'Hotels'

    #Recycling
    tags = {"amenity":"recycling"}
    try:
        recycling = ox.features_from_place(location, tags, which_result=None, buffer_dist=None).reset_index()
    except:
        try:
            recycling = ox.features_from_point(geolocate(location),tags,6000)
        except:
            recycling = gpd.GeoDataFrame()
    if 'geometry' in recycling.columns:
        recycling['geometry'] = recycling['geometry'].apply(lambda geom: geom if geom.type=='Point' else geom.centroid)

        recycling['nearest_node'] = ox.nearest_nodes(osmG,recycling.geometry.x,recycling.geometry.y)
    recycling['Type'] = 'Recycling'
    #Urban gardens
    tags = {"landuse":"allotments"}
    try:
        urb_gardens = ox.features_from_place(location, tags, which_result=None, buffer_dist=None).reset_index()
    except:
        try:
            urb_gardens = ox.features_from_point(geolocate(location),tags,6000)
        except:
            urb_gardens = gpd.GeoDataFrame()
    if 'geometry' in urb_gardens.columns:
        urb_gardens['geometry'] = urb_gardens['geometry'].apply(lambda geom: geom if geom.type=='Point' else geom.centroid)

        urb_gardens['nearest_node'] = ox.nearest_nodes(osmG,urb_gardens.geometry.x,urb_gardens.geometry.y)
    urb_gardens['Type'] = 'Urban Gardens'
    
    if known_location:
        optimal_point = gpd.GeoSeries(Point(reversed(geolocate(location))))
    else:
        optimal_point = hotels.dissolve().centroid

    optimal_point.name = 'geometry'
    optimal_point = gpd.GeoDataFrame(optimal_point,crs=4326)
    optimal_point['nearest_node'] = ox.nearest_nodes(osmG,optimal_point.geometry.x,optimal_point.geometry.y)

    hotels['shortest_path'] = hotels['nearest_node'].apply(
        lambda origin: ox.shortest_path(osmG,origin,optimal_point['nearest_node'][0])
    )

    hotels['shortest_path_edges'] = hotels['shortest_path'].apply(shortest_path_edges)

    hotels['shortest_path_edges'] = hotels['shortest_path'].apply(shortest_path_edges)

    hotelpaths = hotels.explode('shortest_path_edges')

    hotelpaths = hotelpaths[~hotelpaths.shortest_path_edges.isna()]
    hotelpaths = hotelpaths[~hotelpaths.shortest_path_edges.apply(lambda x: len(x)==0)]

    hotelpaths['u'] = hotelpaths['shortest_path_edges'].apply(lambda x: x[0])
    hotelpaths['v'] = hotelpaths['shortest_path_edges'].apply(lambda x: x[1])

    hotelpaths['count'] = 1

    hotelpaths = hotelpaths.groupby(['u','v']).agg({'count':'count'}).reset_index()

    gdf_nodes, gdf_edges = ox.graph_to_gdfs(osmG)

    gdf_edges = gdf_edges.reset_index().drop_duplicates(['u','v'])
    gdf_edges = gdf_edges.merge(hotelpaths, left_on=['u','v'],right_on=['u','v'],how='left')[['u','v','geometry','count']].fillna(0)

    gdf_edges['width'] = normalize_column(gdf_edges['count'])
    print(location)
    
    pois = gpd.GeoDataFrame(pd.concat([hotels, recycling, urb_gardens],ignore_index=True, axis=0)[['Type','geometry']])
    
    return optimal_point, pois, gdf_edges
#     except:
#         print('error:', location)
#         return ()

In [5]:
df = pd.read_excel('../03. Temporal/COP_TRACKER.xlsx')
for col in ['HostCoords','shortest_path','shortest_path_edges']:
    df[col] = df[col].apply(eval)
for col in ['HostGeometry','GuestAirportGeometry','HostAirportGeometry']:
    df[col] = gpd.GeoSeries.from_wkt(df[col])

#Airports and Routes

airports = gpd.read_file('../Resources/OpenFlights/airports')

# airports = airports[airports['AirportNam'].str.find('International') != -1]

airports.Country = airports.Country.replace(
    {
        "Congo (Brazzaville)":"Congo",
        "Congo (Kinshasa)":"Democratic Republic of the Congo",
        "Cote d'Ivoire":"Côte d'Ivoire",
        "Czech Republic":"Czechia",
        "East Timor":"Timor-Leste",
        "Swaziland":"Eswatini",
        "United States":"United States of America",
        "West Bank":"Palestine",
        "Burma":"Myanmar"
    }
)

airports['AirportGeometry'] = airports['geometry']

airports = airports.rename(columns={'City':'AirportCity', 'Country':'AirportCountry'})

airport_columns = ['AirportID','AirportGeometry','AirportNam','AirportCity','AirportCountry','IATA','ICAO']

routes = gpd.read_file('../Resources/OpenFlights/routes')
routes['Count'] = 1

routes_grouped = routes.groupby(['SAirportID','DAirportID','SourceCity','DestCity']).agg({'geometry':'first','Count':'count','Distance':'mean'}).reset_index().set_geometry('geometry')
routes_grouped = routes_grouped[routes_grouped['Distance']!=0]

# G = nx.DiGraph()
# G.add_nodes_from(list(airports['AirportID'].unique()))
# G.add_edges_from([list(pair) for pair in list(routes_grouped[['SAirportID','DAirportID']].values)])

G = nx.from_pandas_edgelist(
    routes_grouped,
    source='SAirportID',
    target='DAirportID',
    edge_attr='Distance',
    create_using=nx.DiGraph()
)

G.remove_nodes_from(list(nx.isolates(G)))
# print(len(G.nodes))
airports = airports[airports['AirportID'].isin(list(G.nodes))]

#Join df to shp
df_flights = df.explode('shortest_path_edges')

df_flights['path_index'] = df_flights['shortest_path_edges'].apply(lambda x: x[0] if type(x)==list else np.nan)
df_flights['SAirportID'] = df_flights['shortest_path_edges'].apply(lambda x: x[1][0] if type(x)==list else np.nan)
df_flights['DAirportID'] = df_flights['shortest_path_edges'].apply(lambda x: x[1][1] if type(x)==list else np.nan)

df_flights = gpd.GeoDataFrame(df_flights.merge(routes_grouped[['SAirportID', 'DAirportID', 'Distance', 'geometry']]), crs=4326)

df = df.merge(
    df_flights.groupby(['COP','GuestCountry']).agg({'Distance':'sum','geometry':list}).reset_index(),
    left_on=['COP','GuestCountry'], right_on=['COP','GuestCountry'],how='left'
)

df['geometry'] = df['geometry'].apply(lambda x: MultiLineString(x) if type(x) == list else np.nan)
df['geometry'] = df.apply(lambda row: row['geometry'] if row['geometry'] != np.nan else row['HostGeometry'],axis=1)

df = gpd.GeoDataFrame(df,crs=4326)

df = df[df['GuestCountry']!='European Union']

df['Distance'] = df.apply(lambda row: row['Distance'] if row['GuestCountry']!=row['HostCountry'] else 0,axis=1)

df['Flights'] = df['Assistance'] / PRIVATE_JET_OCUP
df['FlightCO2Em'] = (df['Flights'] * (df['Distance']/1000) * PRIVATE_JET_EF)/1000

In [6]:
countries = df[['GuestCountry']].drop_duplicates()

countries['PopulationModel'] = countries['GuestCountry'].apply(
    lambda country: estimate_ls('GuestCountry',country,df,'Population',['Year'])
)
countries['EmissionsModel'] = countries['GuestCountry'].apply(
    lambda country: estimate_ls('GuestCountry',country,df,'CO2Em/Year',['Year'])
)

countries['AssistanceModel'] = countries['GuestCountry'].apply(
    lambda country: estimate_ls('GuestCountry',country,df,'Assistance',['Year','CO2Em/Year','Distance'])
)

# countries['location'] = countries['GuestCountry'].apply(lambda x: f'{get_capital(x)}, {x}')
# countries['localCOP'] = countries['location'].apply(imagine_local_cop)

In [7]:
countries['PopulationModelR2'] = countries['GuestCountry'].apply(
    lambda country: estimate_ls('GuestCountry',country,df,'Population',['Year'], return_r2=True)
)
countries['EmissionsModelR2'] = countries['GuestCountry'].apply(
    lambda country: estimate_ls('GuestCountry',country,df,'CO2Em/Year',['Year'], return_r2=True)
)

countries['AssistanceModelR2'] = countries['GuestCountry'].apply(
    lambda country: estimate_ls('GuestCountry',country,df,'Assistance',['Year','CO2Em/Year','Distance'], return_r2=True)
)

In [8]:
locations = {'Germany': {'City': 'Bonn', 'Site': 'World Conference Center Bonn (WCCB)'},
 'Poland': {'City': 'Katowice', 'Site': 'International Congress Centre (MCK)'},
 'Spain': {'City': 'Madrid', 'Site': 'Feria de Madrid (IFEMA)'},
 'United Kingdom': {'City': 'Glasgow', 'Site': 'Scottish Event Campus (SEC)'},
 'Egypt': {'City': 'Sharm el-Sheikh','Site': 'International Convention Center, El-Salam Road'},
 'United Arab Emirates': {'City': 'Dubai', 'Site': 'Expo City Dubai'},
 'Azerbaijan': {'City': 'Baku', 'Site': None},
 'Brazil': {'City': 'Belem', 'Site': None}}

In [9]:
# imagine_local_cops={}

# for country in locations:
#     if country not in imagine_local_cops:
#         try:
#             imagine_local_cops[country] = imagine_local_cop(
#                 f'{locations[country]["City"]}, {country}',
#                 known_location=locations[country]['Site']
#             )
#         except:
#             imagine_local_cops[country] = imagine_local_cop(
#                 locations[country]["City"],
#                 known_location=locations[country]['Site']
#             )

In [10]:
# with open('imagine_local_cops.pkl', 'wb') as fp:
#     pickle.dump(imagine_local_cops, fp)
#     print('dictionary saved successfully to file')
    
with open('imagine_local_cops.pkl', 'rb') as fp:
    imagine_local_cops = pickle.load(fp)
    print('imagine_local_cops')

imagine_local_cops


In [11]:
# imagine_local_cops = {}
imagine_local_cops.keys()

dict_keys(['Germany', 'Poland', 'Spain', 'United Kingdom', 'Egypt', 'United Arab Emirates', 'Azerbaijan', 'Brazil'])

In [14]:
#COP29 Azerbaijan and COP30 Belem

cop29 = estimate_cop('COP29',2024,'Azerbaijan','Baku',df,assistance_factor=1, emissions_factor=1, shortest_path_method='min_length')
cop30 = estimate_cop('COP30',2025,'Brazil','Belem',df,assistance_factor=1, emissions_factor=1, shortest_path_method='min_length')
cops_proj = pd.concat([df,cop29,cop30])

#Multicriteria Prioritization
CO2EMPC_COEF = 1
CO2EM_COEF = 6
POP_COEF = 1
ASSIST_COEF = 0
DIST_COEF = 0
FLICO2EM_COEF = 1

cops_proj = apply_prioritization(cops_proj)

SELEC_COP = 'COP28'
VIS_PRIOR = 10

In [31]:
cops_proj_gr = cops_proj[(cops_proj['Hierarchy']<=VIS_PRIOR)].groupby(['COP','Year']).agg({'FlightCO2Em':'sum'}).reset_index().rename(columns={'FlightCO2Em':'Projected'})
cops_proj_gr['Projected'] = cops_proj_gr['Projected'].astype(int)

cops_proj_gr['Historic'] = cops_proj_gr.apply(lambda row: row['Projected'] if row['Year'] <= 2023 else np.nan, axis=1)
cops_proj_gr['Projected'] = cops_proj_gr.apply(lambda row: row['Projected'] if row['Year'] >= 2023 else np.nan, axis=1)

centroid = cops_proj.total_bounds
centroid_x = (centroid[0] + centroid[2]) / 2
centroid_y = (centroid[1] + centroid[3]) / 2
centroid = [centroid_y,centroid_x]

hostdict = cops_proj.groupby('COP').agg({'HostCity':'first','HostCountry':'first', 'Year':'first'}).T.to_dict()
optimal_point, pois, gdf_edges = imagine_local_cops[hostdict[SELEC_COP]['HostCountry']]

centroid_local = optimal_point.total_bounds
centroid_x_local = (centroid_local[0] + centroid_local[2]) / 2
centroid_y_local = (centroid_local[1] + centroid_local[3]) / 2
centroid_local = [centroid_y_local,centroid_x_local]

TOT_KM = "{:.0f} MM km".format(cops_proj[(cops_proj['COP']==SELEC_COP)&(cops_proj['Hierarchy']<=VIS_PRIOR)]['Distance'].sum() / 1000000)
TOT_EMISSIONS = "{:.0f} TON OF CO2".format(cops_proj[(cops_proj['COP']==SELEC_COP)&(cops_proj['Hierarchy']<=VIS_PRIOR)]['FlightCO2Em'].sum())

RAND_IND = random.randint(0,len(emission_comparisson)-1)

EQ_EMISSIONS = "equivalent to {:.0f}".format(
    (cops_proj[(cops_proj['COP']==SELEC_COP)&(cops_proj['Hierarchy']<=VIS_PRIOR)]['FlightCO2Em'].sum())*emission_comparisson[RAND_IND][0]
) + emission_comparisson[RAND_IND][1]

ranking = cops_proj[
    (cops_proj['COP']==SELEC_COP)&(cops_proj['Hierarchy']<=VIS_PRIOR)
].groupby('Hierarchy').agg({'GuestCountry':'first'}).reset_index().rename(
    columns={'Hierarchy':'Pos','GuestCountry':f'{SELEC_COP} Guests'}
)

In [36]:
app = DashProxy(
    transforms=[MultiplexerTransform()], 
    external_stylesheets=[dbc.themes.DARKLY],
    external_scripts=[
        {'src': 'https://cdnjs.cloudflare.com/ajax/libs/html2canvas/1.4.0/html2canvas.min.js'}
    ]
)

app.title= 'COP Tracker ✈️'

SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "20rem",
    "padding": "2rem 1rem",
    "background-color": "#f8f9fa",
    "fontSize":"80%",
    "lineheight":"80%",
    "overflow": "scroll"
}

CONTENT_STYLE = {
    "margin-left": "20rem"
}

colorscale = ['#0b090a', '#161a1d', '#660708', '#a4161a', '#e5383b', '#b1a7a6', '#d3d3d3', '#f5f3f4']

cop_colors = extract_colors('Reds',len(hostdict.keys()),return_rgb=False)
cop_colors = {list(hostdict.keys())[i]:cop_colors[i] for i in range(len(list(hostdict.keys())))}
cop_colors['COP29'] = '#74D0E7'
cop_colors['COP30'] = '#219ebc'

poi_markers = {
    'Hotels':'https://upload.wikimedia.org/wikipedia/commons/thumb/9/92/Location_dot_red.svg/768px-Location_dot_red.svg.png',
    'Recycling':'https://upload.wikimedia.org/wikipedia/commons/thumb/3/35/Location_dot_blue.svg/768px-Location_dot_blue.svg.png',
    'Urban Gardens':'https://upload.wikimedia.org/wikipedia/commons/thumb/9/91/Location_dot_orange.svg/768px-Location_dot_orange.svg.png'
}

basemap = [
    dl.TileLayer(
        url='https://tiles.stadiamaps.com/tiles/alidade_smooth_dark/{z}/{x}/{y}{r}.png',
        attribution='&copy; <a href="https://www.stadiamaps.com/" target="_blank">Stadia Maps</a> &copy; <a href="https://openmaptiles.org/" target="_blank">OpenMapTiles</a> &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors',

    #     url = 'https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}',
    #     attribution = 'Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ',
        maxZoom=16
    ),
]

#PREPARE GLOBAL MAP
temp_map = basemap.copy()

cops_proj[(cops_proj['Hierarchy']<=VIS_PRIOR)&(cops_proj['COP']==SELEC_COP)].explode().reset_index().apply(
        lambda row: temp_map.append(
            dl.Polyline(
                positions=[(coord[1], coord[0]) for coord in row['geometry'].coords],
                color='#D52221',
                weight=2
            )
        ),axis=1
)

#PREPARE LOCAL MAP
temp_map2 = basemap.copy()

gdf_edges[gdf_edges['width']!=0].apply(
    lambda row: temp_map2.append(
        dl.Polyline(
            positions=[(coord[1], coord[0]) for coord in row['geometry'].coords],
            color='#D52221',
            weight=0.2+(row['width']*10)
        )
    ),axis=1
)

optimal_point.geometry.apply(
    lambda geom: temp_map2.append(dl.Marker(
        position=[geom.y,geom.x],
        icon=dict(
            iconUrl='https://upload.wikimedia.org/wikipedia/commons/thumb/5/5e/Dot-white.svg/2048px-Dot-white.svg.png',
            iconSize=[20, 20],
        )
    ))
)

pois.apply(
    lambda row: temp_map2.append(dl.Marker(
        position=[row['geometry'].y,row['geometry'].x],
        icon=dict(
            iconUrl=poi_markers[row['Type']],
            iconSize=[5, 5],
        )
    )),axis=1
)

viewport = dict(center=centroid_local)

#Generate Graph
fig = create_line_chart(cops_proj_gr, x='Year', y=['Historic','Projected'],text='COP')

app.layout = dbc.Container([
    dbc.Row(
        [
            dbc.Row(
                dbc.Label(' COP Tracker ✈️', style={'color':'white'}),
                style={'backgroundColor':colorscale[0],'fontSize':'100%','fontWeight':'bold','padding':'0px 0px 0px 10px'}
            ),
            dbc.Row([
                    dbc.Col([
                        dbc.Label('Choose a COP', style={'color':'white','fontSize':'70%','fontWeight':'bold','padding':'0px 0px 0px 10px'}),
                        dcc.Dropdown(
                            id='checklist-countries',
                            options=[
                                {
                                    'value':cop,
                                    'label':html.Span([f"{cop} ({hostdict[cop]['HostCity']}, {hostdict[cop]['HostCountry']})"],style={'color':'black'}),
                                } for cop in list(hostdict.keys())
                            ],
                            value=SELEC_COP,
                            multi=False,
                            placeholder='COP',
                            style={'fontSize':'90%'}
                        )
                    ], style={'backgroundColor':colorscale[1]},width=4), 
                    dbc.Col([
                        dbc.Label('Most Critical Guests', style={'color':'white','fontSize':'70%','fontWeight':'bold','padding':'0px 0px 0px 10px'}),
                        dcc.Slider(0, len(cops_proj['GuestCountry'].unique()),value=10,id='slider-prior',tooltip={"placement": "bottom", "always_visible": False}),
                    ], style={'backgroundColor':colorscale[1]},width=5),
                    dbc.Col([
                        dbc.Row(html.Br(id="redirect")),
                        dbc.Col(dbc.Button("Go to GitHub Repository",id='github-button'),width={'size': 8, 'offset': 2})
                    ], style={'backgroundColor':colorscale[1]})
                ],style={'padding':'0px 0px 0px 0px'}
            ),
        ],
        className='sticky-top fluid',style={'padding':'0px 0px 0px 10px','backgroundColor':colorscale[0]}
    ),
    dbc.Row([
        dbc.Col(dl.MapContainer(
            id='map-historic',
            children=temp_map,
            style={
                'width': '100%', 
                'height': '600px'
            },
            center=centroid,
            zoom=2
        ),width=4),
        dbc.Col([
            dbc.Row(html.Div(html.Div(id='ranking-table',
            children=dbc.Table.from_dataframe(
                ranking, striped=True, bordered=True, hover=True,style={'fontSize':'10px'},color='dark'
            ),style={'padding-left':'15px',"maxHeight": "375px", "overflow": "scroll"}),style={'height':'380'})),
            html.Br(),
            dbc.Row(dbc.Label(f'The {VIS_PRIOR} Most Critical Guests', id='top-emissions',style={'color':'white','fontSize':'80%','padding-left':'15px','text-align':'center'})),
            dbc.Row(dbc.Label(f'at {SELEC_COP}', id='cop-emissions',style={'color':'white','fontSize':'120%','padding-left':'15px','text-align':'center'})),
            dbc.Row(dbc.Label(f'travelled {TOT_KM}', id='val-km',style={'color':'white','fontSize':'100%','fontWeight':'bold','padding-left':'15px','text-align':'center'})),
            dbc.Row(dbc.Label(f'generating {TOT_EMISSIONS}', id='val-emissions',style={'color':'white','fontSize':'100%','fontWeight':'bold','padding-left':'15px','text-align':'center'})),
            dbc.Row(dbc.Label(EQ_EMISSIONS, id='eq-emissions',style={'color':'white','fontSize':'80%','padding-left':'15px','text-align':'center'})),
        ],width=2),
        dbc.Col(dcc.Graph(
            id='graph-projections',
            figure=fig,
            config={'displayModeBar': False},
            style={
                'width': '100%', 
                'height': '600px',
                'padding-left':'15px',
                'padding-right':'15px',
                'backgroundColor':'#212222'
            }
        ),width=3),
        dbc.Col([
            dbc.Row(dl.MapContainer(
                id='map-local',
                children=temp_map2,
                style={
                    'width': '100%', 
                    'height': '550px'
                },
                center=centroid_local,
                trackViewport=False,
                zoom=12
            )),
            dbc.Row([
                dbc.Col([
                    html.Img(src=poi_markers[poi], height='5px', width='5px', style={'objectFit': 'contain'}),
                    dbc.Label(poi, style={'color': 'white', 'fontSize': '45%'})
                ],style={'height':'15px'},width=2 if poi=='Hotels' else 3) for poi in poi_markers
            ]+[
                dbc.Col([
                    html.Img(src='https://upload.wikimedia.org/wikipedia/commons/thumb/5/5e/Dot-white.svg/2048px-Dot-white.svg.png', height='7px', width='7px', style={'objectFit': 'contain'}),
                    dbc.Label('COP Venue', id='cop-venue-label', style={'color': 'white', 'fontSize': '45%'})
                ],style={'height':'15px'},width=4)
            ],style={'padding':'0px 0px 0px 0px'})  
        ],width=3,style={'backgroundColor':'#212222'}),
    ],className="g-0",style={'backgroundColor':'#212222'}),
],fluid=True,style={'fontFamily':'Montserrat','backgroundColor':'#212222'})

@app.callback(
    [
        Output('map-historic','children'),
        Output('map-local','children'),
        Output('map-local','viewport'),
        Output('cop-venue-label','children'),
        Output('ranking-table','children'),
        Output('top-emissions','children'),
        Output('cop-emissions','children'),
        Output('val-km','children'),
        Output('val-emissions','children'),
        Output('eq-emissions','children'),
        Output('graph-projections','figure')
    ],
    [
        Input('slider-prior','value'),
        Input('checklist-countries','value'),
    ]
)

def update_prioritization(vis_prior, selec_cop):
    global temp_map, temp_map2, SELEC_COP, VIS_PRIOR, optimal_point, pois, gdf_edges, centroid_local, cops_proj_gr

    VIS_PRIOR = vis_prior
    temp_map = basemap.copy()

    cops_proj[(cops_proj['Hierarchy']<=VIS_PRIOR)&(cops_proj['COP']==selec_cop)].explode().reset_index().apply(
            lambda row: temp_map.append(
                dl.Polyline(
                    positions=[(coord[1], coord[0]) for coord in row['geometry'].coords],
                    color='#D52221',
                    weight=2
                )
            ),axis=1
    )
    
    ranking = cops_proj[
        (cops_proj['COP']==selec_cop)&(cops_proj['Hierarchy']<=vis_prior)
    ].groupby('Hierarchy').agg({'GuestCountry':'first'}).reset_index().rename(
        columns={'Hierarchy':'Pos','GuestCountry':f'{selec_cop} Guests'}
    )
    
    ranking = dbc.Table.from_dataframe(
        ranking, striped=True, bordered=True, hover=True,style={'fontSize':'10px'},color='dark'
    )
    
    TOT_KM = "{:.0f} MM km".format(cops_proj[(cops_proj['COP']==selec_cop)&(cops_proj['Hierarchy']<=vis_prior)]['Distance'].sum() / 1000000)
    TOT_EMISSIONS = "{:.0f} ton CO2".format(cops_proj[(cops_proj['COP']==selec_cop)&(cops_proj['Hierarchy']<=vis_prior)]['FlightCO2Em'].sum())
    
    RAND_IND = random.randint(0,len(emission_comparisson)-1)

    EQ_EMISSIONS = "equivalent to {:.0f}".format(
        (cops_proj[(cops_proj['COP']==selec_cop)&(cops_proj['Hierarchy']<=vis_prior)]['FlightCO2Em'].sum())*emission_comparisson[RAND_IND][0]
    ) + emission_comparisson[RAND_IND][1]
    
    cops_proj_gr = cops_proj[(cops_proj['Hierarchy']<=vis_prior)].groupby(['COP','Year']).agg({'FlightCO2Em':'sum'}).reset_index().rename(columns={'FlightCO2Em':'Projected'})
    cops_proj_gr['Projected'] = cops_proj_gr['Projected'].astype(int)

    cops_proj_gr['Historic'] = cops_proj_gr.apply(lambda row: row['Projected'] if row['Year'] <= 2023 else np.nan, axis=1)
    cops_proj_gr['Projected'] = cops_proj_gr.apply(lambda row: row['Projected'] if row['Year'] >= 2023 else np.nan, axis=1)
    
    fig = create_line_chart(cops_proj_gr, x='Year', y=['Historic','Projected'],text='COP')

    if SELEC_COP != selec_cop:
        
        SELEC_COP = selec_cop
        
        optimal_point, pois, gdf_edges = imagine_local_cops[hostdict[SELEC_COP]['HostCountry']]

        centroid_local = optimal_point.total_bounds
        centroid_x_local = (centroid_local[0] + centroid_local[2]) / 2
        centroid_y_local = (centroid_local[1] + centroid_local[3]) / 2
        centroid_local = [centroid_y_local,centroid_x_local]
    
        temp_map2 = basemap.copy()

        gdf_edges[gdf_edges['width']!=0].apply(
            lambda row: temp_map2.append(
                dl.Polyline(
                    positions=[(coord[1], coord[0]) for coord in row['geometry'].coords],
                    color='#D52221',
                    weight=0.2+(row['width']*10)
                )
            ),axis=1
        )

        optimal_point.geometry.apply(
            lambda geom: temp_map2.append(dl.Marker(
                position=[geom.y,geom.x],
                icon=dict(
                    iconUrl='https://upload.wikimedia.org/wikipedia/commons/thumb/5/5e/Dot-white.svg/2048px-Dot-white.svg.png',
                    iconSize=[20, 20],
                )
            ))
        )
        
    pois.apply(
        lambda row: temp_map2.append(dl.Marker(
            position=[row['geometry'].y,row['geometry'].x],
            icon=dict(
                iconUrl=poi_markers[row['Type']],
                iconSize=[5, 5],
            )
        )),axis=1
    )

    viewport = dict(center=centroid_local)
    
    map_text = 'COP Venue' if hostdict[selec_cop]['Year']<2024 else 'COP Venue (Hypothetical)'
    
    num_guests = f'The {vis_prior} Most Critical Guests' if vis_prior <195 else f'The {vis_prior} Guests'
    
    km_travelled = f'travelled {TOT_KM}' if hostdict[selec_cop]['Year']<2024 else f'will travel {TOT_KM}'
    
    return temp_map, temp_map2, viewport, map_text, ranking, num_guests, f'at {selec_cop}', km_travelled, f'generating {TOT_EMISSIONS}', EQ_EMISSIONS, fig


@app.callback(
    Output('redirect', 'children'),
    [Input('github-button', 'n_clicks')]
)
def redirect_to_github(n_clicks):
    if n_clicks is not None:
        webbrowser.open_new_tab("https://github.com/juancgiraldom/cop_tracker")
        return dcc.Location(pathname='/', id='dummy')
    else:
        return 

In [ ]:
server = app.server
if __name__ == "__main__":
    webbrowser.open('http://localhost:8090')
    app.run_server(debug=False, port=8090)